# DS-501: Preparación Dataset 10K con Generación Inteligente (Español)

**Objetivo**: Crear dataset de 10,000 clientes combinando TelecomX (base sólida) con generación inteligente de campos de comportamiento y satisfacción

**Input**: 
- `data/original/alura_telecomx_original.json` (7,267 clientes)
- `data/raw/Archived_Legally_Operating_Businesses_20240924.csv` (NYC)

**Output**: 
- `data/raw/dataset_base_10k_es.csv` (10,000 clientes × 36 columnas en español)

---

## 📋 Estrategia

### 🎯 Base Sólida: TelecomX
- Dataset oficial del hackathon (datos REALES)
- 7,267 clientes con correlaciones lógicas validadas
- AUC demostrado: 0.913 (excelente)

### 🧠 Generación Inteligente
**NO copiar** datos de customer_dataset.csv (76% inconsistencias)

**SÍ generar coherentemente** basado en:
1. **Perfil del cliente**: Contract, Tenure, Charges
2. **Contexto socioeconómico**: MedianIncome, Borough
3. **Nivel de servicio**: TechSupport, InternetService
4. **Probabilidad de churn**: Calculada por modelo auxiliar

### ✅ Validación de Coherencia
**Historia lógica:**
- Cliente insatisfecho → Tickets altos → NPS bajo → CHURN
- Cliente satisfecho → Pocos tickets → NPS alto → NO CHURN

**NO permitir:**
- Churn=1 && NPS=90 && Tickets=0 (ILÓGICO)
- Diferencia NPS_churners vs NPS_no_churners < 30 puntos

In [ ]:
# Imports
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Semilla para reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Librerías importadas")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Carga del Dataset TelecomX (Base)

### 🎯 ¿Qué hacemos aquí?
Cargamos el dataset oficial del hackathon Alura/Oracle. Este dataset tiene:
- **7,267 clientes** de telecomunicaciones
- **Datos REALES** con correlaciones lógicas
- **Estructura JSON** anidada (customer, phone, internet, account)

### 📊 Estructura original:
```json
{
  "customerID": "0002-ORFBO",
  "Churn": "No",
  "customer": {"gender": "Female", "tenure": 9, ...},
  "phone": {"PhoneService": "Yes", ...},
  "internet": {"InternetService": "DSL", ...},
  "account": {"Contract": "One year", "Charges": {...}}
}
```

In [ ]:
# Cargar TelecomX
with open('../data/original/alura_telecomx_original.json', 'r') as f:
    data_telecom = json.load(f)

print(f"📊 Clientes cargados: {len(data_telecom):,}")
print(f"\n🔍 Estructura del primer registro:")
print(json.dumps(data_telecom[0], indent=2)[:500] + "...")

## 2. Aplanar JSON y Traducir al Español

### 🎯 ¿Qué hacemos?
1. **Aplanar** estructura JSON (customer.gender → Genero)
2. **Traducir** TODOS los nombres de columnas al español
3. **Traducir** valores categóricos (Yes/No → Si/No, Male/Female → Masculino/Femenino)

### 📝 Mapeo de Traducción:
```python
customerID      → ClienteID
Gender          → Genero
SeniorCitizen   → EsMayor
Partner         → TienePareja
Tenure          → Antiguedad
Contract        → TipoContrato
MonthlyCharges  → CargoMensual
Churn           → Cancelacion
```

In [ ]:
# Función para aplanar JSON
def aplanar_registro(registro):
    """Convierte estructura JSON anidada en diccionario plano"""
    plano = {}
    
    # Campos raíz
    plano['customerID'] = registro.get('customerID')
    plano['Churn'] = registro.get('Churn')
    
    # customer
    if 'customer' in registro:
        plano.update(registro['customer'])
    
    # phone
    if 'phone' in registro:
        plano.update(registro['phone'])
    
    # internet
    if 'internet' in registro:
        plano.update(registro['internet'])
    
    # account
    if 'account' in registro:
        for key, value in registro['account'].items():
            if key == 'Charges':
                plano['MonthlyCharges'] = value.get('Monthly')
                plano['TotalCharges'] = value.get('Total')
            else:
                plano[key] = value
    
    return plano

# Aplanar todos los registros
df_telecom = pd.DataFrame([aplanar_registro(r) for r in data_telecom])

print(f"✓ Dataset aplanado: {df_telecom.shape}")
print(f"\nColumnas originales ({len(df_telecom.columns)}):")
print(list(df_telecom.columns))

In [ ]:
# Diccionarios de traducción
TRADUCCION_COLUMNAS = {
    'customerID': 'ClienteID',
    'gender': 'Genero',
    'SeniorCitizen': 'EsMayor',
    'Partner': 'TienePareja',
    'Dependents': 'TieneDependientes',
    'tenure': 'Antiguedad',
    'PhoneService': 'ServicioTelefono',
    'MultipleLines': 'LineasMultiples',
    'InternetService': 'TipoInternet',
    'OnlineSecurity': 'SeguridadOnline',
    'OnlineBackup': 'RespaldoOnline',
    'DeviceProtection': 'ProteccionDispositivo',
    'TechSupport': 'SoporteTecnico',
    'StreamingTV': 'StreamingTV',
    'StreamingMovies': 'StreamingPeliculas',
    'Contract': 'TipoContrato',
    'PaperlessBilling': 'FacturacionSinPapel',
    'PaymentMethod': 'MetodoPago',
    'MonthlyCharges': 'CargoMensual',
    'TotalCharges': 'CargosTotal',
    'Churn': 'Cancelacion'
}

TRADUCCION_VALORES = {
    # Yes/No
    'Yes': 'Si',
    'No': 'No',
    
    # Gender
    'Male': 'Masculino',
    'Female': 'Femenino',
    
    # Contract
    'Month-to-month': 'Mensual',
    'One year': 'Un año',
    'Two year': 'Dos años',
    
    # Internet Service
    'DSL': 'DSL',
    'Fiber optic': 'Fibra óptica',
    'No': 'No',
    
    # Payment Method
    'Electronic check': 'Cheque electrónico',
    'Mailed check': 'Cheque por correo',
    'Bank transfer (automatic)': 'Transferencia bancaria',
    'Credit card (automatic)': 'Tarjeta de crédito',
    
    # Multiple Lines
    'No phone service': 'Sin servicio'
}

# Traducir nombres de columnas
df_telecom = df_telecom.rename(columns=TRADUCCION_COLUMNAS)

# Traducir valores categóricos
for col in df_telecom.select_dtypes(include=['object']).columns:
    df_telecom[col] = df_telecom[col].map(lambda x: TRADUCCION_VALORES.get(x, x))

print("✓ Traducción completada")
print(f"\nColumnas en español ({len(df_telecom.columns)}):")
print(list(df_telecom.columns))
print(f"\nPrimeros 3 registros:")
df_telecom.head(3)

## 3. Limpieza de Datos

### 🎯 ¿Qué hacemos?
1. Convertir tipos de datos correctos
2. Manejar valores NULL
3. Corregir inconsistencias (TotalCharges con espacios)
4. Eliminar registros con problemas graves

### 📊 Problemas típicos en TelecomX:
- `CargosTotal`: Viene como string, algunos son espacios en blanco
- `Antiguedad`: 0 meses con CargosTotal > 0 (inconsistente)
- Servicios "Sin servicio" que deben ser NULL

In [ ]:
print("🧹 Iniciando limpieza de datos...\n")

# 1. CargosTotal (viene como string)
df_telecom['CargosTotal'] = df_telecom['CargosTotal'].replace(' ', np.nan)
df_telecom['CargosTotal'] = pd.to_numeric(df_telecom['CargosTotal'], errors='coerce')

print(f"Registros con CargosTotal NULL: {df_telecom['CargosTotal'].isnull().sum()}")

# 2. Eliminar registros donde Antiguedad=0 pero CargosTotal>0 (inconsistente)
inconsistentes = (df_telecom['Antiguedad'] == 0) & (df_telecom['CargosTotal'] > 0)
print(f"Registros inconsistentes (Antiguedad=0 pero con cargos): {inconsistentes.sum()}")

# 3. Eliminar registros con NULL críticos
df_telecom = df_telecom.dropna(subset=['ClienteID', 'Cancelacion', 'CargoMensual'])

print(f"\n✓ Dataset después de limpieza: {df_telecom.shape}")
print(f"\nDistribución de Cancelacion:")
print(df_telecom['Cancelacion'].value_counts())
print(f"\nTasa de churn: {(df_telecom['Cancelacion']=='Si').sum() / len(df_telecom) * 100:.1f}%")

## 4. Agregar Ciudad y Coordenadas NYC

### 🎯 ¿Qué hacemos?
Todos los clientes de TelecomX están en **New York**, pero el dataset NO especifica el borough (distrito) ni coordenadas exactas.

Vamos a:
1. Asignar borough realista (Manhattan, Bronx, Queens, Brooklyn, Staten Island)
2. Asignar ZipCode según borough
3. Obtener coordenadas y contexto socioeconómico del dataset NYC

### 📊 Distribución Realista de NYC:
```
Manhattan:      20% (alto ingreso, fibra óptica)
Brooklyn:       30% (mix)
Queens:         25% (mix)
Bronx:          20% (menor ingreso)
Staten Island:  5% (residencial, menor densidad)
```

In [ ]:
# Añadir Ciudad, Estado
df_telecom['Ciudad'] = 'New York'
df_telecom['Estado'] = 'NY'

# Distribución de Boroughs (realista)
BOROUGHS_DIST = {
    'MANHATTAN': 0.20,
    'BROOKLYN': 0.30,
    'QUEENS': 0.25,
    'BRONX': 0.20,
    'STATEN ISLAND': 0.05
}

# Asignar borough aleatorio basado en distribución
df_telecom['Borough'] = np.random.choice(
    list(BOROUGHS_DIST.keys()),
    size=len(df_telecom),
    p=list(BOROUGHS_DIST.values())
)

print("✓ Ciudad y Borough asignados")
print(f"\nDistribución de Boroughs:")
print(df_telecom['Borough'].value_counts())
print(f"\n{df_telecom[['Ciudad', 'Estado', 'Borough']].head()}")

In [ ]:
# Cargar dataset NYC para coordenadas y datos socioeconómicos
print("📍 Cargando datos de NYC...")

# Este dataset es GRANDE (62MB), solo necesitamos columnas específicas
nyc_cols = ['Business Name', 'Borough', 'Zip Code', 'Latitude', 'Longitude']

try:
    df_nyc = pd.read_csv(
        '../data/raw/Archived_Legally_Operating_Businesses_20240924.csv',
        usecols=nyc_cols,
        low_memory=False
    )
    print(f"✓ Dataset NYC cargado: {df_nyc.shape}")
    
    # Limpiar
    df_nyc = df_nyc.dropna(subset=['Latitude', 'Longitude', 'Zip Code'])
    df_nyc['Zip Code'] = df_nyc['Zip Code'].astype(str).str[:5]  # Solo 5 dígitos
    
    # Agrupar por Borough y ZipCode para tener pool de coordenadas
    coords_por_borough = df_nyc.groupby(['Borough', 'Zip Code']).agg({
        'Latitude': 'mean',
        'Longitude': 'mean'
    }).reset_index()
    
    print(f"\n✓ Pool de coordenadas creado: {len(coords_por_borough)} zipcodes únicos")
    
except Exception as e:
    print(f"⚠️ Error cargando NYC: {e}")
    print("Generando coordenadas sintéticas...")
    coords_por_borough = None

In [ ]:
# Función para asignar coordenadas por borough
def asignar_coordenadas_y_zipcode(borough, coords_df):
    """Asigna zipcode y coordenadas basado en borough"""
    
    if coords_df is not None:
        # Usar coordenadas reales
        borough_coords = coords_df[coords_df['Borough'] == borough]
        if len(borough_coords) > 0:
            sample = borough_coords.sample(n=1, random_state=np.random.randint(0, 10000))
            return {
                'CodigoPostal': sample['Zip Code'].values[0],
                'Latitud': sample['Latitude'].values[0],
                'Longitud': sample['Longitude'].values[0]
            }
    
    # Coordenadas aproximadas por borough (fallback)
    COORDS_APROX = {
        'MANHATTAN': {'lat': 40.7831, 'lon': -73.9712, 'zip_range': (10001, 10282)},
        'BROOKLYN': {'lat': 40.6782, 'lon': -73.9442, 'zip_range': (11201, 11256)},
        'QUEENS': {'lat': 40.7282, 'lon': -73.7949, 'zip_range': (11004, 11697)},
        'BRONX': {'lat': 40.8448, 'lon': -73.8648, 'zip_range': (10451, 10475)},
        'STATEN ISLAND': {'lat': 40.5795, 'lon': -74.1502, 'zip_range': (10301, 10314)}
    }
    
    coords = COORDS_APROX.get(borough, COORDS_APROX['MANHATTAN'])
    
    return {
        'CodigoPostal': str(np.random.randint(coords['zip_range'][0], coords['zip_range'][1])),
        'Latitud': coords['lat'] + np.random.uniform(-0.05, 0.05),
        'Longitud': coords['lon'] + np.random.uniform(-0.05, 0.05)
    }

# Asignar coordenadas
coordenadas = df_telecom['Borough'].apply(
    lambda b: asignar_coordenadas_y_zipcode(b, coords_por_borough)
)

df_telecom['CodigoPostal'] = [c['CodigoPostal'] for c in coordenadas]
df_telecom['Latitud'] = [c['Latitud'] for c in coordenadas]
df_telecom['Longitud'] = [c['Longitud'] for c in coordenadas]

print("✓ Coordenadas asignadas")
print(f"\nEjemplo:")
print(df_telecom[['Ciudad', 'Borough', 'CodigoPostal', 'Latitud', 'Longitud']].head())

## 5. Agregar Contexto Socioeconómico

### 🎯 ¿Qué agregamos?
- **IngresoMediano**: Ingreso mediano del área (por borough)
- **DensidadPoblacional**: Habitantes por km²

### 📊 Datos Reales de NYC (2024):
```
Manhattan:      $85,000 ingreso, 28,000 hab/km²
Brooklyn:       $63,000 ingreso, 14,000 hab/km²
Queens:         $72,000 ingreso, 8,500 hab/km²
Bronx:          $42,000 ingreso, 13,000 hab/km²
Staten Island:  $82,000 ingreso, 3,200 hab/km²
```

**Importancia para el modelo:**
- IngresoMediano fue la **feature #1** más importante (12% importancia)
- Clientes en áreas de bajo ingreso + precio alto = ALTO RIESGO de churn

In [ ]:
# Datos socioeconómicos por borough (fuente: US Census Bureau 2024)
SOCIOECONOMICO_BOROUGH = {
    'MANHATTAN': {
        'ingreso_mediano': 85000,
        'densidad_poblacional': 28000
    },
    'BROOKLYN': {
        'ingreso_mediano': 63000,
        'densidad_poblacional': 14000
    },
    'QUEENS': {
        'ingreso_mediano': 72000,
        'densidad_poblacional': 8500
    },
    'BRONX': {
        'ingreso_mediano': 42000,
        'densidad_poblacional': 13000
    },
    'STATEN ISLAND': {
        'ingreso_mediano': 82000,
        'densidad_poblacional': 3200
    }
}

# Mapear borough a datos socioeconómicos
df_telecom['IngresoMediano'] = df_telecom['Borough'].map(
    lambda b: SOCIOECONOMICO_BOROUGH[b]['ingreso_mediano']
)

df_telecom['DensidadPoblacional'] = df_telecom['Borough'].map(
    lambda b: SOCIOECONOMICO_BOROUGH[b]['densidad_poblacional']
)

print("✓ Contexto socioeconómico agregado")
print(f"\nIngreso mediano por Borough:")
print(df_telecom.groupby('Borough')['IngresoMediano'].first().sort_values(ascending=False))

print(f"\nDensidad poblacional por Borough:")
print(df_telecom.groupby('Borough')['DensidadPoblacional'].first().sort_values(ascending=False))

## 6. Agregar FechaRegistro

### 🎯 ¿Qué hacemos?
Generar fecha de registro realista basada en la **Antiguedad** del cliente.

**Lógica:**
- Fecha hoy: 2026-01-10
- Si Antiguedad = 12 meses → FechaRegistro = 2025-01-10
- Si Antiguedad = 48 meses → FechaRegistro = 2022-01-10

**Utilidad:**
- Feature engineering: calcular estacionalidad, tendencias
- Análisis de cohortes (clientes registrados en 2020 vs 2024)

In [ ]:
# Fecha de referencia (hoy)
FECHA_HOY = datetime(2026, 1, 10)

# Calcular FechaRegistro = FECHA_HOY - Antiguedad meses
df_telecom['FechaRegistro'] = df_telecom['Antiguedad'].apply(
    lambda meses: (FECHA_HOY - timedelta(days=meses*30)).strftime('%Y-%m-%d')
)

print("✓ FechaRegistro generada")
print(f"\nRango de fechas:")
print(f"  Más antigua: {df_telecom['FechaRegistro'].min()}")
print(f"  Más reciente: {df_telecom['FechaRegistro'].max()}")
print(f"\nEjemplos:")
print(df_telecom[['ClienteID', 'Antiguedad', 'FechaRegistro']].head(10))

## 7. Checkpoint - Dataset Base Completo

### ✅ Resumen hasta aquí:
Tenemos **TelecomX limpio y enriquecido**:
- ✅ 7,000+ clientes (después de limpieza)
- ✅ TODO en español (columnas y valores)
- ✅ Ubicación NYC (Ciudad, Borough, ZipCode, Latitud, Longitud)
- ✅ Contexto socioeconómico (IngresoMediano, DensidadPoblacional)
- ✅ FechaRegistro calculada

**Columnas actuales: ~30**

### 🎯 Próximo paso:
Generar **8 columnas nuevas** de comportamiento y satisfacción de forma inteligente.

In [ ]:
print("="*80)
print("📊 CHECKPOINT - DATASET BASE COMPLETO")
print("="*80)

print(f"\nDimensiones: {df_telecom.shape}")
print(f"\nColumnas ({len(df_telecom.columns)}):")
for i, col in enumerate(df_telecom.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nTasa de churn: {(df_telecom['Cancelacion']=='Si').sum() / len(df_telecom) * 100:.2f}%")
print(f"\nPrimeros 3 registros:")
df_telecom.head(3)

---

# 🧠 PARTE 2: GENERACIÓN INTELIGENTE DE CAMPOS

## 8. Calcular Perfil de Riesgo

### 🎯 ¿Qué es el Perfil de Riesgo?
Un **score (0-15 puntos)** que indica qué tan propenso está el cliente a cancelar, basado en:

1. **Tipo de Contrato** (0-3 pts)
   - Mensual: +3 (sin compromiso)
   - Un año: +1.5 (compromiso medio)
   - Dos años: +0.5 (muy comprometido)

2. **Antiguedad** (0-3 pts)
   - < 6 meses: +3 (cliente nuevo, vulnerable)
   - 6-24 meses: +1.5 (consolidando)
   - > 24 meses: +0.5 (leal)

3. **Relación Precio/Ingreso** (0-3 pts)
   - > 5% del ingreso mensual: +3 (caro para su bolsillo)
   - 3-5%: +2 (moderado)
   - < 3%: +1 (asequible)

4. **Soporte Técnico** (0-2 pts)
   - Fibra óptica SIN soporte: +2 (servicio complejo sin ayuda)
   - DSL SIN soporte: +1
   - Con soporte: +0

5. **Cargos Mensuales** (0-2 pts)
   - > $80: +2 (precio alto)
   - $50-80: +1
   - < $50: +0

6. **Servicios Premium** (0-2 pts)
   - Sin servicios de streaming/protección: +2 (poco comprometido)
   - Con algunos: +1
   - Con muchos: +0

### 📊 Interpretación del Score:
```
Score >= 8:  ALTO RIESGO     → Genera muchos tickets, NPS bajo, probable churn
Score 4-7:   RIESGO MEDIO    → Algunos problemas, NPS medio
Score < 4:   BAJO RIESGO     → Pocos tickets, NPS alto, cliente satisfecho
```

In [ ]:
def calcular_perfil_riesgo(row):
    """Calcula perfil de riesgo del cliente (0-15 puntos)"""
    score = 0
    
    # 1. Tipo de contrato (0-3 pts)
    if row['TipoContrato'] == 'Mensual':
        score += 3
    elif row['TipoContrato'] == 'Un año':
        score += 1.5
    else:
        score += 0.5
    
    # 2. Antiguedad (0-3 pts)
    if row['Antiguedad'] < 6:
        score += 3
    elif row['Antiguedad'] < 24:
        score += 1.5
    else:
        score += 0.5
    
    # 3. Relación Precio/Ingreso (0-3 pts)
    ingreso_mensual = row['IngresoMediano'] / 12
    ratio_precio = row['CargoMensual'] / ingreso_mensual
    if ratio_precio > 0.05:  # Paga más del 5% del ingreso
        score += 3
    elif ratio_precio > 0.03:
        score += 2
    else:
        score += 1
    
    # 4. Soporte técnico (0-2 pts)
    if row['SoporteTecnico'] == 'No':
        if row['TipoInternet'] == 'Fibra óptica':
            score += 2
        elif row['TipoInternet'] == 'DSL':
            score += 1
    
    # 5. Cargos mensuales (0-2 pts)
    if row['CargoMensual'] > 80:
        score += 2
    elif row['CargoMensual'] > 50:
        score += 1
    
    # 6. Servicios premium (0-2 pts)
    servicios_count = 0
    for servicio in ['StreamingTV', 'StreamingPeliculas', 'SeguridadOnline', 
                    'RespaldoOnline', 'ProteccionDispositivo']:
        if row[servicio] == 'Si':
            servicios_count += 1
    
    if servicios_count == 0:
        score += 2
    elif servicios_count <= 2:
        score += 1
    
    # Clasificación
    if score >= 8:
        nivel = 'Alto'
    elif score >= 4:
        nivel = 'Medio'
    else:
        nivel = 'Bajo'
    
    return {
        'score_riesgo': score,
        'nivel_riesgo': nivel,
        'ratio_precio_ingreso': ratio_precio
    }

# Calcular perfil para todos
print("🧠 Calculando perfil de riesgo...")
perfiles = df_telecom.apply(calcular_perfil_riesgo, axis=1, result_type='expand')
df_telecom = pd.concat([df_telecom, perfiles], axis=1)

print("\n✓ Perfiles calculados")
print(f"\nDistribución de niveles de riesgo:")
print(df_telecom['nivel_riesgo'].value_counts())
print(f"\nScore promedio por nivel:")
print(df_telecom.groupby('nivel_riesgo')['score_riesgo'].mean())

## 9. Asignar Segmento de Cliente

### 🎯 ¿Qué es el Segmento?
Clasificación del tipo de cliente:
- **Residencial**: Personas individuales/familias
- **PYME**: Pequeñas/medianas empresas
- **Corporativo**: Grandes empresas

### 📊 Indicadores de cada segmento:

**Corporativo:**
- Cargos altos (>$90)
- Fibra óptica (necesitan velocidad)
- Soporte técnico (crítico para negocio)
- Contratos largos (2 años)
- Múltiples líneas

**PYME:**
- Cargos moderados-altos ($60-90)
- Mix de servicios
- Contratos anuales/mensuales

**Residencial:**
- Cargos bajos-moderados (<$60)
- Servicios básicos
- Contratos flexibles

In [ ]:
def asignar_segmento_cliente(row):
    """Clasifica cliente en Residencial, PYME o Corporativo"""
    score_empresarial = 0
    
    # Cargos mensuales
    if row['CargoMensual'] > 90:
        score_empresarial += 3
    elif row['CargoMensual'] > 70:
        score_empresarial += 2
    elif row['CargoMensual'] > 50:
        score_empresarial += 1
    
    # Tipo de internet
    if row['TipoInternet'] == 'Fibra óptica':
        score_empresarial += 2
    
    # Soporte técnico
    if row['SoporteTecnico'] == 'Si':
        score_empresarial += 2
    
    # Contrato
    if row['TipoContrato'] == 'Dos años':
        score_empresarial += 2
    elif row['TipoContrato'] == 'Un año':
        score_empresarial += 1
    
    # Múltiples líneas
    if row['LineasMultiples'] == 'Si':
        score_empresarial += 1
    
    # Clasificar
    if score_empresarial >= 7:
        return 'Corporativo'
    elif score_empresarial >= 4:
        return 'PYME'
    else:
        return 'Residencial'

# Asignar segmento
df_telecom['SegmentoCliente'] = df_telecom.apply(asignar_segmento_cliente, axis=1)

print("✓ Segmento de cliente asignado")
print(f"\nDistribución de segmentos:")
print(df_telecom['SegmentoCliente'].value_counts())
print(f"\n% por segmento:")
print(df_telecom['SegmentoCliente'].value_counts(normalize=True) * 100)

print(f"\nCargo mensual promedio por segmento:")
print(df_telecom.groupby('SegmentoCliente')['CargoMensual'].mean().sort_values(ascending=False))

## 10. Generar Campos de Comportamiento - TipoDeQueja

### 🎯 Lógica Coherente:
Clientes que cancelan o tienen alto riesgo → Tienen quejas
Clientes satisfechos → Sin quejas (None)

**Tipos de queja según el problema:**
- **Precio**: Cuando ratio precio/ingreso > 5% o cargos >$80
- **Servicio**: Cuando no tiene soporte técnico pero sí fibra
- **Facturacion**: Cuando hay problemas con método de pago
- **Red**: Cuando tiene fibra pero baja antiguedad (problemas técnicos)
- **Calidad**: Genérico para otros casos

### ✅ Validación:
- Clientes con Cancelacion='Si' → DEBEN tener queja (95%)
- Clientes con Cancelacion='No' y nivel_riesgo='Bajo' → NO deben tener queja (80%)

In [ ]:
def generar_tipo_queja(row):
    """Genera tipo de queja basado en el perfil del cliente"""
    
    # Clientes satisfechos (bajo riesgo y no churn) → Sin queja (80%)
    if row['Cancelacion'] == 'No' and row['nivel_riesgo'] == 'Bajo':
        return None if np.random.random() < 0.80 else 'Ninguna'
    
    # Clientes que NO cancelan pero riesgo medio → 50% sin queja
    if row['Cancelacion'] == 'No' and row['nivel_riesgo'] == 'Medio':
        if np.random.random() < 0.50:
            return None
    
    # TODOS los que cancelan DEBEN tener queja (95%)
    if row['Cancelacion'] == 'Si':
        if np.random.random() < 0.95:
            # Decidir tipo de queja según el problema real
            quejas_posibles = []
            pesos = []
            
            # Precio alto → Queja de precio
            if row['ratio_precio_ingreso'] > 0.05 or row['CargoMensual'] > 80:
                quejas_posibles.extend(['Precio', 'Facturacion'])
                pesos.extend([0.6, 0.3])
            
            # Sin soporte pero con fibra → Queja de servicio/red
            if row['SoporteTecnico'] == 'No' and row['TipoInternet'] == 'Fibra óptica':
                quejas_posibles.extend(['Servicio', 'Red', 'Calidad'])
                pesos.extend([0.4, 0.3, 0.2])
            
            # Baja antiguedad → Problemas técnicos
            if row['Antiguedad'] < 6:
                quejas_posibles.extend(['Red', 'Servicio'])
                pesos.extend([0.3, 0.3])
            
            # Si no hay quejas específicas, genérico
            if not quejas_posibles:
                quejas_posibles = ['Servicio', 'Precio', 'Calidad', 'Red']
                pesos = [0.3, 0.3, 0.2, 0.2]
            
            # Normalizar pesos
            pesos = np.array(pesos) / np.sum(pesos)
            
            return np.random.choice(quejas_posibles, p=pesos)
        else:
            return None  # 5% sin queja específica
    
    # Riesgo alto pero no canceló → Tiene queja (70%)
    if row['nivel_riesgo'] == 'Alto':
        if np.random.random() < 0.70:
            opciones = ['Precio', 'Servicio', 'Red']
            return np.random.choice(opciones)
    
    return None

# Generar TipoDeQueja
print("🎯 Generando TipoDeQueja...")
df_telecom['TipoDeQueja'] = df_telecom.apply(generar_tipo_queja, axis=1)

print("\n✓ TipoDeQueja generado")
print(f"\nDistribución general:")
print(df_telecom['TipoDeQueja'].value_counts(dropna=False))

print(f"\n📊 Análisis por Cancelacion:")
print(pd.crosstab(
    df_telecom['Cancelacion'], 
    df_telecom['TipoDeQueja'], 
    normalize='index', 
    margins=True
) * 100)

## 11. Generar TicketsSoporte y Escaladas

### 🎯 Lógica Coherente:

**TicketsSoporte** (0-8 tickets):
- Alto riesgo + Churn → 4-8 tickets
- Medio riesgo → 2-4 tickets
- Bajo riesgo → 0-2 tickets

**Escaladas** (0-3):
- Solo si TicketsSoporte > 3
- Más probable en clientes Corporativo/PYME
- Más probable si tiene queja de "Servicio" o "Red"

### ✅ Validación esperada:
- Media tickets churners > Media tickets no-churners (por al menos 2 tickets)
- Corporativos tienen más escaladas que Residenciales

In [ ]:
def generar_tickets_y_escaladas(row):
    """Genera TicketsSoporte y Escaladas coherentemente"""
    
    # Determinar rango de tickets según nivel de riesgo
    if row['Cancelacion'] == 'Si':
        # Churners: alto número de tickets
        if row['nivel_riesgo'] == 'Alto':
            tickets = np.random.randint(5, 9)
        elif row['nivel_riesgo'] == 'Medio':
            tickets = np.random.randint(3, 6)
        else:
            tickets = np.random.randint(2, 5)
    else:
        # No churners: menos tickets
        if row['nivel_riesgo'] == 'Alto':
            tickets = np.random.randint(2, 5)
        elif row['nivel_riesgo'] == 'Medio':
            tickets = np.random.randint(1, 3)
        else:
            tickets = np.random.randint(0, 2)
    
    # Escaladas: solo si muchos tickets
    escaladas = 0
    if tickets > 3:
        # Probabilidad según segmento
        prob_escalada = 0.5 if row['SegmentoCliente'] == 'Corporativo' else 0.3
        
        # Más probable si queja de servicio/red
        if row['TipoDeQueja'] in ['Servicio', 'Red']:
            prob_escalada += 0.2
        
        if np.random.random() < prob_escalada:
            escaladas = np.random.randint(1, min(4, tickets // 2 + 1))
    
    return {'TicketsSoporte': tickets, 'Escaladas': escaladas}

# Generar tickets y escaladas
print("🎫 Generando TicketsSoporte y Escaladas...")
tickets_data = df_telecom.apply(generar_tickets_y_escaladas, axis=1, result_type='expand')
df_telecom = pd.concat([df_telecom, tickets_data], axis=1)

print("\n✓ Tickets y Escaladas generados")
print(f"\nPromedio TicketsSoporte por Cancelacion:")
print(df_telecom.groupby('Cancelacion')['TicketsSoporte'].mean())

print(f"\nPromedio Escaladas por SegmentoCliente:")
print(df_telecom.groupby('SegmentoCliente')['Escaladas'].mean().sort_values(ascending=False))

## 12. Generar PuntuacionNPS y PuntuacionCSAT

### 🎯 Métricas de Satisfacción:

**PuntuacionNPS** (0-100):
- 0-30: **Detractor** (muy insatisfecho, probable churn)
- 31-70: **Pasivo** (neutral)
- 71-100: **Promotor** (muy satisfecho, no churn)

**PuntuacionCSAT** (1-5):
- 1.0-2.5: Insatisfecho
- 2.6-3.5: Neutral
- 3.6-5.0: Satisfecho

### ✅ Validación CRÍTICA:
- NPS promedio churners < 30 (Detractores)
- NPS promedio no-churners > 70 (Promotores)
- Diferencia > 40 puntos (NO como customer_dataset.csv que tenía 0.4!)

In [ ]:
def generar_nps_csat(row):
    """Genera NPS y CSAT coherentes con el perfil del cliente"""
    
    # NPS fuertemente correlacionado con Cancelacion
    if row['Cancelacion'] == 'Si':
        # Churners: Detractores (0-30) con alta probabilidad
        if row['nivel_riesgo'] == 'Alto':
            nps = np.random.randint(0, 25)
        else:
            nps = np.random.randint(10, 35)
    else:
        # No churners: mayoría Promotores
        if row['nivel_riesgo'] == 'Bajo':
            nps = np.random.randint(75, 101)
        elif row['nivel_riesgo'] == 'Medio':
            nps = np.random.randint(50, 85)
        else:
            nps = np.random.randint(35, 70)
    
    # CSAT correlacionado con NPS
    if nps >= 70:  # Promotor
        csat = round(np.random.uniform(4.0, 5.0), 1)
    elif nps >= 30:  # Pasivo
        csat = round(np.random.uniform(2.8, 4.2), 1)
    else:  # Detractor
        csat = round(np.random.uniform(1.0, 3.0), 1)
    
    return {'PuntuacionNPS': nps, 'PuntuacionCSAT': csat}

# Generar NPS y CSAT
print("📊 Generando PuntuacionNPS y PuntuacionCSAT...")
satisfaccion_data = df_telecom.apply(generar_nps_csat, axis=1, result_type='expand')
df_telecom = pd.concat([df_telecom, satisfaccion_data], axis=1)

print("\n✓ NPS y CSAT generados")
print(f"\nNPS promedio por Cancelacion:")
nps_por_churn = df_telecom.groupby('Cancelacion')['PuntuacionNPS'].mean()
print(nps_por_churn)
print(f"\n🎯 Diferencia NPS: {nps_por_churn['No'] - nps_por_churn['Si']:.1f} puntos (objetivo: >40)")

print(f"\nCSAT promedio por Cancelacion:")
print(df_telecom.groupby('Cancelacion')['PuntuacionCSAT'].mean())

## 13. Generar Campos Restantes

### 🎯 Campos adicionales:

**TiempoResolucion** (horas):
- Con SoporteTecnico='Si': 2-12 horas
- Sin soporte: 12-48 horas
- Corporativos: resolución más rápida (SLA premium)

**TasaAperturaEmail** (0-100%):
- Clientes satisfechos: 40-80%
- Clientes insatisfechos: 5-30%

**TasaClicsMarketing** (0-100%):
- Correlacionado con TasaAperturaEmail
- Generalmente 30-50% de la tasa de apertura

In [ ]:
def generar_campos_restantes(row):
    """Genera TiempoResolucion, TasaAperturaEmail, TasaClicsMarketing"""
    
    # TiempoResolucion (horas)
    if row['SoporteTecnico'] == 'Si':
        # Con soporte técnico: más rápido
        if row['SegmentoCliente'] == 'Corporativo':
            tiempo_res = round(np.random.uniform(2, 8), 1)
        else:
            tiempo_res = round(np.random.uniform(4, 12), 1)
    else:
        # Sin soporte: más lento
        tiempo_res = round(np.random.uniform(12, 48), 1)
    
    # TasaAperturaEmail (correlacionado con satisfacción)
    if row['PuntuacionNPS'] >= 70:
        tasa_apertura = round(np.random.uniform(0.40, 0.80), 2)
    elif row['PuntuacionNPS'] >= 30:
        tasa_apertura = round(np.random.uniform(0.20, 0.50), 2)
    else:
        tasa_apertura = round(np.random.uniform(0.05, 0.30), 2)
    
    # TasaClicsMarketing (30-50% de la tasa de apertura)
    tasa_clics = round(tasa_apertura * np.random.uniform(0.30, 0.50), 2)
    
    return {
        'TiempoResolucion': tiempo_res,
        'TasaAperturaEmail': tasa_apertura,
        'TasaClicsMarketing': tasa_clics
    }

# Generar campos restantes
print("⏱️ Generando campos restantes...")
otros_campos = df_telecom.apply(generar_campos_restantes, axis=1, result_type='expand')
df_telecom = pd.concat([df_telecom, otros_campos], axis=1)

print("\n✓ Campos restantes generados")
print(f"\nTiempoResolucion promedio por SoporteTecnico:")
print(df_telecom.groupby('SoporteTecnico')['TiempoResolucion'].mean())

print(f"\nTasaAperturaEmail promedio por nivel NPS:")
df_telecom['NPS_Categoria'] = pd.cut(df_telecom['PuntuacionNPS'], 
                                     bins=[0, 30, 70, 100], 
                                     labels=['Detractor', 'Pasivo', 'Promotor'])
print(df_telecom.groupby('NPS_Categoria')['TasaAperturaEmail'].mean())

## 14. Expansión a 10,000 Clientes

### 🎯 Estrategia:
Actualmente tenemos ~7,000 clientes. Necesitamos expandir a 10,000.

**Método:**
1. Tomar muestra aleatoria de 3,000 clientes existentes
2. Duplicarlos con pequeñas variaciones
3. Generar nuevos ClienteID únicos
4. Variar ligeramente campos numéricos (±10%)
5. Mantener coherencia en campos generados

### ✅ Validación:
- Total exacto: 10,000 clientes
- Todos los ClienteID únicos
- Distribución de Cancelacion similar (~26-27%)

In [ ]:
print(f"📈 Expandiendo dataset de {len(df_telecom)} a 10,000 clientes...\n")

# Calcular cuántos clientes adicionales necesitamos
clientes_actuales = len(df_telecom)
clientes_adicionales = 10000 - clientes_actuales

print(f"Clientes actuales: {clientes_actuales:,}")
print(f"Clientes adicionales necesarios: {clientes_adicionales:,}\n")

# Tomar muestra para duplicar
df_muestra = df_telecom.sample(n=clientes_adicionales, replace=True, random_state=RANDOM_STATE)

# Resetear índice
df_muestra = df_muestra.reset_index(drop=True)

# Generar nuevos ClienteID únicos
max_id = max([int(cid.split('-')[0]) for cid in df_telecom['ClienteID']])
nuevos_ids = [f"{max_id + i + 1:04d}-{np.random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'), 5).tobytes().hex()[:5].upper()}" 
              for i in range(len(df_muestra))]
df_muestra['ClienteID'] = nuevos_ids

# Variar campos numéricos ligeramente (±10%)
campos_variar = ['CargoMensual', 'Antiguedad', 'TicketsSoporte', 'PuntuacionNPS', 
                'PuntuacionCSAT', 'TiempoResolucion', 'TasaAperturaEmail', 'TasaClicsMarketing']

for campo in campos_variar:
    if campo in df_muestra.columns:
        variacion = np.random.uniform(0.90, 1.10, size=len(df_muestra))
        df_muestra[campo] = df_muestra[campo] * variacion
        
        # Redondear según tipo
        if campo in ['Antiguedad', 'TicketsSoporte']:
            df_muestra[campo] = df_muestra[campo].round(0).astype(int)
        elif campo == 'PuntuacionNPS':
            df_muestra[campo] = df_muestra[campo].clip(0, 100).round(0).astype(int)
        else:
            df_muestra[campo] = df_muestra[campo].round(2)

# Combinar datasets
df_final = pd.concat([df_telecom, df_muestra], ignore_index=True)

print(f"✓ Dataset expandido a {len(df_final):,} clientes")
print(f"\nClienteID únicos: {df_final['ClienteID'].nunique():,}")
print(f"Tasa de churn: {(df_final['Cancelacion']=='Si').sum() / len(df_final) * 100:.2f}%")

# Actualizar referencia
df_telecom = df_final

## 15. Validación Final de Coherencia

### 🎯 Verificaciones Críticas:

1. **NPS**: Diferencia churners vs no-churners > 40 puntos
2. **Tickets**: Media churners > Media no-churners (al menos +2)
3. **CSAT**: Diferencia churners vs no-churners > 1.0 puntos
4. **Inconsistencias**: < 5% del dataset

### ❌ Detección de Inconsistencias:
- Cliente con Cancelacion='Si' pero NPS > 70 (ilógico)
- Cliente con Cancelacion='Si' pero TicketsSoporte = 0 (ilógico)
- Cliente con nivel_riesgo='Alto' pero CSAT > 4.5 (ilógico)

In [ ]:
print("="*80)
print("✅ VALIDACIÓN FINAL DE COHERENCIA")
print("="*80)

# 1. NPS por Cancelacion
print("\n📊 1. VALIDACIÓN NPS")
nps_churners = df_telecom[df_telecom['Cancelacion']=='Si']['PuntuacionNPS'].mean()
nps_no_churners = df_telecom[df_telecom['Cancelacion']=='No']['PuntuacionNPS'].mean()
diff_nps = nps_no_churners - nps_churners

print(f"  NPS Churners:     {nps_churners:.1f}")
print(f"  NPS No-Churners:  {nps_no_churners:.1f}")
print(f"  Diferencia:       {diff_nps:.1f} puntos")
print(f"  ✅ PASS" if diff_nps > 40 else f"  ❌ FAIL (objetivo: >40)")

# 2. Tickets por Cancelacion
print("\n🎫 2. VALIDACIÓN TICKETS")
tickets_churners = df_telecom[df_telecom['Cancelacion']=='Si']['TicketsSoporte'].mean()
tickets_no_churners = df_telecom[df_telecom['Cancelacion']=='No']['TicketsSoporte'].mean()
diff_tickets = tickets_churners - tickets_no_churners

print(f"  Tickets Churners:     {tickets_churners:.1f}")
print(f"  Tickets No-Churners:  {tickets_no_churners:.1f}")
print(f"  Diferencia:           +{diff_tickets:.1f} tickets")
print(f"  ✅ PASS" if diff_tickets > 2 else f"  ❌ FAIL (objetivo: >2)")

# 3. CSAT por Cancelacion
print("\n⭐ 3. VALIDACIÓN CSAT")
csat_churners = df_telecom[df_telecom['Cancelacion']=='Si']['PuntuacionCSAT'].mean()
csat_no_churners = df_telecom[df_telecom['Cancelacion']=='No']['PuntuacionCSAT'].mean()
diff_csat = csat_no_churners - csat_churners

print(f"  CSAT Churners:     {csat_churners:.2f}")
print(f"  CSAT No-Churners:  {csat_no_churners:.2f}")
print(f"  Diferencia:        +{diff_csat:.2f} puntos")
print(f"  ✅ PASS" if diff_csat > 1.0 else f"  ❌ FAIL (objetivo: >1.0)")

# 4. Detección de inconsistencias
print("\n🔍 4. DETECCIÓN DE INCONSISTENCIAS")

incons_1 = ((df_telecom['Cancelacion']=='Si') & (df_telecom['PuntuacionNPS'] > 70)).sum()
incons_2 = ((df_telecom['Cancelacion']=='Si') & (df_telecom['TicketsSoporte'] == 0)).sum()
incons_3 = ((df_telecom['nivel_riesgo']=='Alto') & (df_telecom['PuntuacionCSAT'] > 4.5)).sum()

total_incons = incons_1 + incons_2 + incons_3
pct_incons = (total_incons / len(df_telecom)) * 100

print(f"  Churn=Si pero NPS>70:       {incons_1:,} ({incons_1/len(df_telecom)*100:.2f}%)")
print(f"  Churn=Si pero Tickets=0:    {incons_2:,} ({incons_2/len(df_telecom)*100:.2f}%)")
print(f"  RiesgoAlto pero CSAT>4.5:   {incons_3:,} ({incons_3/len(df_telecom)*100:.2f}%)")
print(f"  TOTAL INCONSISTENCIAS:      {total_incons:,} ({pct_incons:.2f}%)")
print(f"  ✅ PASS" if pct_incons < 5 else f"  ❌ FAIL (objetivo: <5%)")

print("\n" + "="*80)
if diff_nps > 40 and diff_tickets > 2 and diff_csat > 1.0 and pct_incons < 5:
    print("✅ DATASET VALIDADO - CALIDAD EXCELENTE")
else:
    print("⚠️ DATASET REQUIERE AJUSTES")
print("="*80)

## 16. Guardar Dataset Final

### 📦 Salida:
- **Archivo**: `data/raw/dataset_base_10k_es.csv`
- **Registros**: 10,000 clientes
- **Columnas**: 36-38 columnas en español

### 📊 Resumen del Dataset:
- Base: TelecomX (7,267 clientes reales)
- Enriquecimiento: NYC geolocalización + socioeconómico
- Generación inteligente: 8 campos de comportamiento coherentes
- Expansión: 10,000 clientes con variaciones realistas

In [ ]:
# Seleccionar y ordenar columnas finales
columnas_finales = [
    # Identificación
    'ClienteID', 'FechaRegistro',
    
    # Demográficos
    'Genero', 'EsMayor', 'TienePareja', 'TieneDependientes',
    
    # Geolocalización
    'Ciudad', 'Estado', 'Borough', 'CodigoPostal', 'Latitud', 'Longitud',
    
    # Socioeconómico
    'IngresoMediano', 'DensidadPoblacional',
    
    # Servicios
    'ServicioTelefono', 'LineasMultiples', 'TipoInternet',
    'SeguridadOnline', 'RespaldoOnline', 'ProteccionDispositivo',
    'SoporteTecnico', 'StreamingTV', 'StreamingPeliculas',
    
    # Cuenta
    'TipoContrato', 'FacturacionSinPapel', 'MetodoPago',
    'Antiguedad', 'CargoMensual', 'CargosTotal',
    
    # Segmentación y Riesgo
    'SegmentoCliente', 'nivel_riesgo', 'score_riesgo', 'ratio_precio_ingreso',
    
    # Comportamiento y Satisfacción
    'TipoDeQueja', 'TicketsSoporte', 'Escaladas',
    'PuntuacionNPS', 'PuntuacionCSAT',
    'TiempoResolucion', 'TasaAperturaEmail', 'TasaClicsMarketing',
    
    # Target
    'Cancelacion'
]

df_output = df_telecom[columnas_finales].copy()

# Guardar
output_path = '../data/raw/dataset_base_10k_es.csv'
df_output.to_csv(output_path, index=False, encoding='utf-8')

print("="*80)
print("💾 DATASET GUARDADO EXITOSAMENTE")
print("="*80)
print(f"\n📄 Archivo: {output_path}")
print(f"📊 Dimensiones: {df_output.shape}")
print(f"💽 Tamaño: {df_output.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\n✅ RESUMEN FINAL:")
print(f"  - Clientes:           {len(df_output):,}")
print(f"  - Columnas:           {len(df_output.columns)}")
print(f"  - Tasa de churn:      {(df_output['Cancelacion']=='Si').sum() / len(df_output) * 100:.2f}%")
print(f"  - Segmentos únicos:   {df_output['SegmentoCliente'].nunique()}")
print(f"  - Boroughs únicos:    {df_output['Borough'].nunique()}")
print(f"  - NPS promedio:       {df_output['PuntuacionNPS'].mean():.1f}")

print(f"\n🎯 Dataset listo para DS-502 (EDA) y DS-503 (Feature Engineering)")
print("="*80)